# Topic Modelling
Topic modelling (konu modelleme), büyük veri setlerinde metinler üzerinde çalışırken metinlerin konusunu tespit etmemizi sağlayan bir yaklaşımdır. Bu yaklaşımda, en yaygını *Latent Dirichlet Allocation (LDA) olmak üzere, NMF (Non-Negative Matrix Factorization), LSA (Latent Semantic Analysis), PLSA (Probabilistic latent Semantic Analysis)* gibi teknikler kullanılır.

### Kullanılacak verilerin elde edilmesi;

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import string
from wordcloud import STOPWORDS

#NLTK Libraries
import nltk
nltk.download('words')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import remove_stopwords


# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\edouc\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
def data_extraction(urls):
    texts = []
    for link in urls:
        txt = ''
        response = requests.get(link)
        content = response.content
        soup = BeautifulSoup(response.content, 'html.parser')
        for t in soup.find_all('p'):
            txt += t.get_text()
        texts.append(txt)
    return texts

def clear_text(texts):
    documents = []

    lemmatizer = WordNetLemmatizer()
    
    for item in range(0, len(texts)):
        
        document = texts[item].encode("ascii", errors="ignore").decode()
        
        # Remove all the special characters
        document = re.sub(r'\W', ' ', document)
    
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        
        # Remove single characters from the start
        document = re.sub(r'\b[a-zA-Z]\b', ' ', document)
        #document = re.sub(r'(?:^| )\w(?:$| )', ' ', document)
        
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
        
        # Removing numbers
        document = re.sub(r'[0-9]', '', document)
        
        # Converting to Lowercase
        document = document.lower()
        
        # Lemmatization
        document = document.split()
    
        document = [lemmatizer.lemmatize(word) for word in document]
        document = ' '.join(document)
        tokenized_doc = word_tokenize(document)
        tokens_without_sw = [word for word in tokenized_doc if word not in STOPWORDS]
        words = set(nltk.corpus.words.words())
        final_doc = [word for word in tokens_without_sw if word in words]
        
        documents.append(tokens_without_sw)
        
    return documents

def build_btgram(documents):

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(documents, min_count=1, threshold=2, delimiter = b'_') # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[documents], min_count=1, threshold=2, delimiter = b'_')  
    
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    def bigrams(texts):
        return [bigram_mod[doc] for doc in texts]
    
    def trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]
    
    item = bigrams(documents)
    item = trigrams(documents)
    for sent in [tokens_]:
        ttokens_ = trigram_phraser[sent]
    

    return documents

**Ben bu çalışmada, National Geographic websitesi üzerinden alınmış beş farklı konudaki makaleler üzerinde çalıştım. Bu makaleler;**

**- Storm Surges**

**- Green Sea Turtle**

**- Climate Change & Wildfires**

**- Plastic Pollution**

**- Supermoon**

**konularından oluşuyor. Dolayısıyla bu üç yöntemle elde edeceğimiz konuların bu şekilde olmasını bekliyoruz.**

In [3]:
urls = ['https://www.nationalgeographic.com/environment/article/storm-surges',
        'https://kids.nationalgeographic.com/animals/reptiles/facts/green-sea-turtle',
        'https://www.nationalgeographic.com/science/article/climate-change-increases-risk-fires-western-us',
        'https://www.nationalgeographic.com/science/article/plastic-pollution-huge-problem-not-too-late-to-fix-it',
        'https://kids.nationalgeographic.com/space/article/what-is-a-supermoon']

In [4]:
document = data_extraction(urls)
document = clear_text(document)


# LDA (Latent Dirichlet Allocation)
Topic modelling uygulamasında en yaygın kullanılan yöntemdir. Her belgenin farklı konuların koleksiyonundan ve her bir konunun farklı kelimelerin koleksiyonundan meydana geldiği fikrini temel alan, en basit kabul edilen bir konu modelleme örneğidir. LDA algoritması kullanıldığında sonuç olarak konuları belirleyen kelimeleri ve yüzdelerini elde ederiz. Python üzerinde uygulaması şu şekilde;

Döküman üzerinden bir sözlük ve o sözlük ile bir corpus oluşturuyoruz;

In [5]:
# Create Dictionary
id2word = corpora.Dictionary(document)
# Create Corpus
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in document]

Sonrasında gensim kütüphanesini kullanarak bir LDA modeli oluşturuyoruz. Bu modeli oluştururken kullandığımız *num_topics* parametresi kaç tane konu belirleneceğini ayarladığımız parametredir. **Coherence(tutarlılık) score** değerini en yüksek olarak alabildiğimiz topic sayısını önceden kontrol ederek bu değeri bu data seti için 50 olarak atadım. **Perplexity** değeri ise yine modelin ne kadar başarılı olduğunu gösteren bir parametredir. Bu değer ne kadar düşük olursa model o kadar iyi demektir.

In [9]:
from pprint import pprint
# number of topics
num_topics = 50
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=document, dictionary=id2word)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

[(5,
  '0.014*"fire" + 0.012*"plastic" + 0.009*"climate" + 0.007*"say" + '
  '0.007*"change" + 0.007*"risk" + 0.006*"waste" + 0.006*"heat" + 0.006*"ha" + '
  '0.005*"million"'),
 (35,
  '0.011*"fire" + 0.008*"plastic" + 0.006*"say" + 0.006*"climate" + '
  '0.005*"change" + 0.005*"heat" + 0.004*"ha" + 0.004*"across" + 0.004*"waste" '
  '+ 0.004*"risk"'),
 (41,
  '0.032*"plastic" + 0.009*"waste" + 0.009*"say" + 0.007*"new" + '
  '0.006*"million" + 0.006*"will" + 0.006*"production" + 0.006*"fire" + '
  '0.005*"climate" + 0.005*"study"'),
 (33,
  '0.018*"plastic" + 0.010*"fire" + 0.009*"say" + 0.008*"climate" + '
  '0.008*"waste" + 0.006*"year" + 0.005*"ha" + 0.005*"new" + 0.005*"change" + '
  '0.004*"will"'),
 (19,
  '0.023*"storm" + 0.018*"surge" + 0.015*"water" + 0.014*"hurricane" + '
  '0.009*"plastic" + 0.008*"wind" + 0.007*"ocean" + 0.005*"cause" + '
  '0.005*"national" + 0.005*"wa"'),
 (6,
  '0.013*"plastic" + 0.009*"fire" + 0.007*"say" + 0.006*"climate" + '
  '0.006*"waste" + 0.006

Topic-keyword dağılımını daha net görebilmek için kaynak olarak yararlandığım internet sitesinde önerilen pyLDAvis ile gösterim yaptım. Burada büyük dairelere baktığımızda konu dağılımını daha net görebiliriz.

In [10]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
30    -0.033502  0.107633       1        1  31.622148
27    -0.048008 -0.010188       2        1  23.589198
25     0.223612 -0.065515       3        1  16.325187
22    -0.051405 -0.034888       4        1  10.788850
45    -0.092581 -0.248038       5        1   9.936398
47    -0.057707 -0.014796       6        1   3.830755
28     0.003698  0.025863       7        1   2.790845
40    -0.028171  0.014824       8        1   0.963310
35    -0.014418  0.015460       9        1   0.003650
29     0.205695 -0.036277      10        1   0.003650
48    -0.009713  0.025421      11        1   0.003650
31    -0.005708 -0.025036      12        1   0.003650
32    -0.030865 -0.005489      13        1   0.003650
33    -0.024766  0.014163      14        1   0.003650
34    -0.010308 -0.041888      15        1   0.003650
36     0.015854  0.019895      16        1   0.003650
46    -0.025272  0.027313      17        1   0.003650
37    -0.003567 -0.013302      18        1   0.003650
26     0.015359 -0.019624      19        1   0.003650
39    -0.006718  0.039931      20        1   0.003650
41    -0.036079  0.002814      21        1   0.003650
42     0.035978  0.034170      22        1   0.003650
43    -0.033546  0.042673      23        1   0.003650
44    -0.048345  0.045212      24        1   0.003650
38    -0.030740 -0.034730      25        1   0.003650
0     -0.000787 -0.021764      26        1   0.003650
1     -0.025363  0.069687      27        1   0.003650
24    -0.016234  0.012392      28        1   0.003650
2     -0.022125  0.046404      29        1   0.003650
3      0.007364  0.029361      30        1   0.003650
4      0.059168  0.002689      31        1   0.003650
5     -0.032071  0.043116      32        1   0.003650
6     -0.022206  0.013804      33        1   0.003650
7      0.079924 -0.026506      34        1   0.003650
8     -0.031333 -0.012731      35        1   0.003650
9      0.028638 -0.022111      36        1   0.003650
10     0.024657  0.029293      37        1   0.003650
11    -0.022008 -0.000026      38        1   0.003650
12    -0.040432 -0.037104      39        1   0.003650
13    -0.069591 -0.109195      40        1   0.003650
14    -0.009404  0.049950      41        1   0.003650
15     0.108176  0.013257      42        1   0.003650
16    -0.009031  0.004323      43        1   0.003650
17    -0.012927  0.026700      44        1   0.003650
18    -0.014004 -0.014308      45        1   0.003650
19     0.084343 -0.012446      46        1   0.003650
20    -0.018881  0.038700      47        1   0.003650
21     0.057104  0.004024      48        1   0.003650
23    -0.008975  0.008459      49        1   0.003650
49    -0.002779 -0.001570      50        1   0.003650, topic_info=         Term       Freq      Total Category  logprob  loglift
1111  plastic  49.000000  49.000000  Default  30.0000  30.0000
488      fire  25.000000  25.000000  Default  29.0000  29.0000
193     storm  24.000000  24.000000  Default  28.0000  28.0000
197     surge  20.000000  20.000000  Default  27.0000  27.0000
222     water  20.000000  20.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
72       even   0.000166   7.776719  Topic50  -6.4308  -0.5335
230      will   0.000188  13.184871  Topic50  -6.3102  -0.9409
311     plant   0.000156   6.098775  Topic50  -6.4930  -0.3527
219        wa   0.000168   9.622408  Topic50  -6.4225  -0.7382
501    global   0.000153   7.383936  Topic50  -6.5133  -0.5642

[3449 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
353       1  0.523550  accelerate
355       1  0.215098   according
355       2  0.430197   according
355       4  0.215098   according
793       2  0.514630     account
...     ...       ...         ...
350       2  0.269294        year
350       4  0.067323        year
350       5  0.

Burada ise her bir metnin konusunu oluşturan ana keyword'leri net bir şekilde gösteren tabloyu oluşturdum. 

In [11]:
def format_topics_sentences(model=lda_model, corpus=corpus, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(model[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(model=lda_model, corpus=corpus, texts=document)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Topic Keywords', 'Text']

# Show
df_dominant_topic.head(5)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Topic Keywords,Text
0,0,25.0,0.9974,"storm, surge, hurricane, water, wind, inland, ...","[wind, make, hurricane, threat, storm, create,..."
1,1,45.0,0.9956,"sea, turtle, green, egg, nesting, shell, will,...","[green, sea, turtle, world, largest, specie, h..."
2,2,30.0,0.8716,"fire, climate, risk, ha, heat, change, across,...","[heating, planet, ha, driven, huge, increase, ..."
3,3,27.0,0.5944,"plastic, waste, new, ton, production, million,...","[correcting, plastic, waste, problem, requires..."
4,4,47.0,0.9880,"moon, time, earth, supermoon, plastic, nationa...","[find, make, moon, appear, extra, big, bright,..."


**Yukarıdaki tabloda, elde ettiğimiz topic keyword'lerin, üzerinde çalıştığımız makalelerle örtüştüğünü ve bu yöntemde başarılı bir sonuç elde ettiğimizi söyleyebiliriz.**

# LSA (Latent Semantic Analysis)


Aynı zamanda LSI(Latent Semantic Index) olarak da bilinir. LSA'nın amacı, sınıflandırma için boyutu azaltmaktır. Bu yöntemdeki temel fikir, sahip olduğumuz document-term matrisini document-topic matrisi ve topic-term matrisi olarak ikiye parçalamaktır. Ancak bunu yaparken her bir kelimenin belgelerdeki bulunma sayısı(raw count) yerine cosine similarity konusunda kullandığımız **tf-idf** değeri kullanılır. LSA genel olarak bir dimension reduction veya noise reducing tekniği olarak kullanılır. Bunu yaparken ise kelimeler ve belgeler arasındaki ilişkileri yakalayan topic keyword'leri bulabilmek için SVD(singular value decomposition) yöntemini kullanır. Bu bileşenler, gensim kütüphanesinin sağladığı LsiModel içerisinde bulunur.

Anlaması ve uygulaması en kolay metoddur. Diğer algoritmalara göre daha hızlıdır çünkü yalnızca decomposition işlemi uygular. Ancak bunun yanında kelimelerin farklı anlamlarını tespit etme konusunda başarısızdır. LDA'ya göre daha zordur ve daha az doğru sonuçlar verir.

Uygulamasını yaptığımızda ise, topic keyword'lerinin yine üzerinde çalıştığımız dökümanlarla örtüştüğünü görüyoruz.

In [12]:
from gensim.models import LsiModel
document = data_extraction(urls)
document = clear_text(document)

# Create Dictionary
id2word = corpora.Dictionary(document)
# Create Corpus
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in document]

# generate LSA model
lsamodel = LsiModel(corpus, num_topics=15, id2word = id2word)  # train model
pprint(lsamodel.print_topics(num_topics=15, num_words=10))

# Compute Coherence Score
coherence_model_lsa = CoherenceModel(model=lsamodel, texts=document, dictionary=id2word)
coherence_lsa = coherence_model_lsa.get_coherence()
print('\nCoherence Score: ', coherence_lsa)

[(0,
  '-0.607*"plastic" + -0.243*"waste" + -0.175*"say" + -0.174*"new" + '
  '-0.139*"million" + -0.134*"fire" + -0.121*"production" + -0.115*"will" + '
  '-0.114*"year" + -0.110*"ton"'),
 (1,
  '-0.430*"fire" + 0.325*"plastic" + -0.211*"climate" + -0.158*"heat" + '
  '-0.152*"risk" + -0.151*"ha" + -0.143*"forest" + -0.143*"change" + '
  '0.130*"waste" + -0.116*"across"'),
 (2,
  '0.565*"storm" + 0.475*"surge" + 0.316*"hurricane" + 0.234*"water" + '
  '0.158*"wind" + 0.131*"ocean" + 0.113*"inland" + 0.090*"wall" + '
  '0.090*"create" + 0.088*"damage"'),
 (3,
  '-0.584*"sea" + -0.523*"turtle" + -0.370*"green" + -0.155*"egg" + '
  '-0.123*"nesting" + -0.092*"shell" + -0.081*"will" + -0.063*"distance" + '
  '-0.062*"nest" + -0.062*"life"'),
 (4,
  '-0.528*"moon" + -0.301*"earth" + -0.301*"supermoon" + -0.296*"time" + '
  '-0.151*"brighter" + -0.151*"full" + -0.151*"appear" + -0.149*"look" + '
  '-0.148*"larger" + -0.147*"month"')]

Coherence Score:  0.5693386761751025


# NMF (Non-Negative Matrix Factorization)

Yüksek boyutlu vektörlerin, çarpanlarına ayrılarak düşük boyutlu vektörler haline getirilmesini temel alan yöntemdir. Ancak adından da anlaşılacağı üzere içerdiği vektörlerin negatif olmaması gerekmektedir. Elimizdeki ana matrisi(X) kullanarak, X = WH olacak şekilde üç matris elde etmiş oluyoruz. Bu matrisler;

**- (X)Document-Word Matrix: Hangi belgelerde hangi kelimelerin göründüğü bilgisini içerir**

**- (W)Basis Vectors: Belgeler üzerinden elde edilen konu kümeleri**

**- (H)Coefficient Matrix: Her bir belgedeki konuların ağırlıklarının bulunduğu matris**

şeklindedir. Topic modelling konusunda bu yöntemi kullanırken ise her bir belge için kelimelere anlamlarına göre bir ağırlık atanmasını sağlıyoruz ve ağırlığı en yüksek kelimeler kümesinin, belgenin konusunu gösterdiğini düşünerek hareket ediyoruz. 

In [13]:
document = data_extraction(urls)
documents = clear_text(document)

# Create Dictionary
id2word = corpora.Dictionary(documents)
# Create Corpus
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in documents]

In [15]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
# use tfidf by removing tokens that don't appear in at least 50 documents
vect = TfidfVectorizer(stop_words='english')
 
# Fit and transform
X = vect.fit_transform(document)
nmf = NMF(n_components=5, random_state=5)
nmf.fit(X)
W = nmf.transform(X)
H = nmf.components_

# Create a DataFrame: components_df
components_df = pd.DataFrame(H, columns=vect.get_feature_names())

D:\Anaconda\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [190]:
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'Topic {topic+1}:')
    print(tmp.nlargest(10))
    print('\n')

Topic 1:
plastic          0.897475
waste            0.438765
new              0.257450
plastics         0.239327
production       0.219383
tons             0.199439
million          0.176997
says             0.144815
billion          0.139607
environmental    0.139607
Name: 0, dtype: float64


Topic 2:
sea          0.625461
green        0.395028
turtles      0.345265
eggs         0.131676
nesting      0.131676
turtle       0.131676
adult        0.065838
color        0.065838
distances    0.065838
female       0.065838
Name: 1, dtype: float64


Topic 3:
storm         0.555103
surges        0.313754
surge         0.193079
hurricane     0.168945
hurricanes    0.168945
water         0.163166
winds         0.144810
inland        0.120675
ocean         0.116831
create        0.096540
Name: 2, dtype: float64


Topic 4:
moon         0.571886
earth        0.326792
supermoon    0.326792
appear       0.163396
brighter     0.163396
14           0.131827
looks        0.131827
time         0.116789


## Kaynaklar;

-https://www.nationalgeographic.com/environment/article/storm-surges

-https://kids.nationalgeographic.com/animals/reptiles/facts/green-sea-turtle

-https://www.nationalgeographic.com/science/article/climate-change-increases-risk-fires-western-us

-https://www.nationalgeographic.com/science/article/plastic-pollution-huge-problem-not-too-late-to-fix-it

-https://kids.nationalgeographic.com/space/article/what-is-a-supermoon




-https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

-https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

-https://medium.com/@busragokmen67/latent-dirichlet-allocation-lda-kullanarak-nas%C4%B1l-topic-modeling-yap%C4%B1l%C4%B1r-75fe8dddcdd2



-https://medium.com/nanonets/topic-modeling-with-lsa-psla-lda-and-lda2vec-555ff65b0b05

-https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

-https://forestforthetree.com/statistics/2018/01/28/topic-modelling-with-lsa-and-lda.html

-https://towardsdatascience.com/2-latent-methods-for-dimension-reduction-and-topic-modeling-20ff6d7d547



-https://medium.com/ml2vec/topic-modeling-is-an-unsupervised-learning-approach-to-clustering-documents-to-discover-topics-fdfbf30e27df

-https://medium.com/voice-tech-podcast/topic-modelling-using-nmf-2f510d962b6e

-https://predictivehacks.com/topic-modelling-with-nmf-in-python/

-https://towardsdatascience.com/topic-modeling-articles-with-nmf-8c6b2a227a45
